In [ ]:
#Importando as bibliotecas
from jupyter_dash import JupyterDash
import os 
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import numpy as np
import urllib.request

#Lendo Arquivo
url = 'https://raw.githubusercontent.com/lalz93/Data_FV/master/SE_Data.csv'
df = pd.read_csv(url, error_bad_lines=False,sep =',')
df = pd.DataFrame(df)
df['DateTime'] = pd.to_datetime(df["DateTime"])
df = df.set_index('DateTime')
del df['Unnamed: 0']

#Criando colunas e df para os intervalos de horários
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Months'] = df.index.month_name()
df['Day'] = df.index.day
df['Days'] = df.index.day_name()
df['Hour'] = df.index.hour
df['Minute'] = df.index.minute
df['Date'] = df.index.date
df['Time'] = df.index.time
df['datetime'] = df.index.to_pydatetime()
df['Month-Year'] = df['Month'].map(str) + '-' + df['Year'].map(str)
year = df.index.year
month = df.index.month
months = df.index.month_name()
day = df.index.day
days = df.index.day_name()
hour = df.index.hour
minute = df.index.minute

#Filtrando as horas exclusivas
hours = hour.unique().tolist()
hours.sort()

#Filtrando os minutos exclusivos
minutes = minute.unique().tolist()
minutes.sort()

#Filtrando a potência
energy_series = df["Pac(W)"]

#Gráfico da potência real ao longo do tempo
energy_data = go.Scatter(x=energy_series.index,
                        y=energy_series.values,
                        line=go.scatter.Line(color='blue', width = 0.6),
                           opacity=0.8,
                           name='energy',
                           text=[f'energy: {x:.0f} kWh' for x in energy_series.values])

layout = go.Layout(height=600, width=1300, font=dict(size=12),
                   title='Solar Energy Real Data with Range Selection',
                   xaxis=dict(title='Date',
                                        # Range selector with buttons
                                         rangeselector=dict(
                                             # Buttons for selecting time scale
                                             buttons=list([
                                                 # 4 hours
                                                 dict(count=4,
                                                      label='4h',
                                                      step='hour',
                                                      stepmode='backward'),
                                                 # 12 hours
                                                 dict(count=12,
                                                      label='12h',
                                                      step='hour',
                                                      stepmode='backward'),
                                                # 1 day
                                                 dict(count=1,
                                                      label='1d',
                                                      step='day',
                                                      stepmode='todate'),
                                                # 1 week
                                                 dict(count=7,
                                                      label='1w',
                                                      step='day',
                                                      stepmode='todate'),
                                                 # 1 month
                                                 dict(count=1,
                                                      label='1m',
                                                      step='month',
                                                      stepmode='backward'),
                                                # 6 months
                                                 dict(count=6,
                                                      label='6m',
                                                      step='month',
                                                      stepmode='backward'),
                                                 # 1 year
                                                 dict(count=1,
                                                      label='1y',
                                                      step='year',
                                                      stepmode='backward'),
                                                # YTD
                                                 dict(count=1,
                                                      label='YTD',
                                                      step='year',
                                                      stepmode='todate'),
                                                  # Entire scale
                                                 dict(step='all')
                                             ])
                                         ),
                                         # Sliding for selecting time window
                                         rangeslider=dict(visible=True),
                                         # Type of xaxis
                                         type='date'),
                   # yaxis is unchanged
                   yaxis=dict(title='Power(kW)')
                   )          
fig = go.Figure(data=[energy_data], layout=layout)

#Dados para gerar os gráficos da potência total, média e Boxplot
a = df.loc['2019']
b = df.loc['2020']

data_columns = ['Pac(W)']
ac = a[data_columns].resample('M').sum().reset_index()
ac = ac.set_index('DateTime')
ac['Month'] = ac.index.month

data_columns = ['Pac(W)']
bc = b[data_columns].resample('M').sum().reset_index()
bc = bc.set_index('DateTime')
bc['Month'] = bc.index.month

data_columns = ['Pac(W)']
cd = a[data_columns].resample('M').mean().reset_index()
cd = cd.set_index('DateTime')
cd['Month'] = cd.index.month

data_columns = ['Pac(W)']
de = b[data_columns].resample('M').mean().reset_index()
de = de.set_index('DateTime')
de['Month'] = de.index.month

layout2 = go.Layout(height=600, width=1300, barmode='group', xaxis=dict(title='Months'), yaxis=dict(title='Power(kW)'))

#Gráfico potência total
figPT = go.Figure(data=[
    go.Bar(name='2019', x = ac["Month"],  y=ac["Pac(W)"], marker_color='rgb(47,138,196)'),
    go.Bar(name='2020', x = bc["Month"],  y=bc["Pac(W)"], marker_color='rgb(158,185,243)')
], layout = layout2)

#Gráfico potência média
figPM = go.Figure(data=[
    go.Bar(name='2019', x = cd["Month"],  y=cd["Pac(W)"], marker_color='rgb(47,138,196)'),
    go.Bar(name='2020', x = de["Month"],  y=de["Pac(W)"], marker_color='rgb(158,185,243)')
], layout = layout2)

#Gráfico BoxPlot - Mensal

figBM = make_subplots(rows=2, cols=1, shared_xaxes=True)
figBM.append_trace((go.Box (name='2019',x=a["Month"],y=a["Pac(W)"], marker_color='rgb(47,138,196)')), row=1, col=1)

figBM.append_trace((go.Box (name='2020',x=b["Month"],y=b["Pac(W)"], marker_color='rgb(158,185,243)')), row=2, col=1)

figBM.update_layout(height=600, width=1300)

#Gráfico Boxplot - Dias da Semana
figBD = make_subplots(rows=2, cols=1, shared_xaxes=True)

figBD.append_trace((go.Box (name='2019',x=a["Days"],y=a["Pac(W)"], marker_color='rgb(47,138,196)')), row=1, col=1)

figBD.append_trace((go.Box (name='2020',x=b["Days"],y=b["Pac(W)"],  marker_color='rgb(158,185,243)')), row=2, col=1)

figBD.update_layout(height=600, width=1300)

#Dados para gerar a curva típica - 2019
m1 = df.iloc[(year == 2019) & (month == 1)]
gm1 = m1[['Pac(W)']].groupby([m1['Time']]).mean()
gm1 = gm1.reset_index()

m2 = df.iloc[(year == 2019) & (month == 2)]
gm2 = m2[['Pac(W)']].groupby([m2['Time']]).mean()
gm2 = gm2.reset_index()

m3 = df.iloc[(year == 2019) & (month == 3)]
gm3 = m3[['Pac(W)']].groupby([m1['Time']]).mean()
gm3 = gm3.reset_index()

m4 = df.iloc[(year == 2019) & (month == 4)]
gm4 = m4[['Pac(W)']].groupby([m4['Time']]).mean()
gm4 = gm4.reset_index()

m5 = df.iloc[(year == 2019) & (month == 5)]
gm5 = m5[['Pac(W)']].groupby([m5['Time']]).mean()
gm5 = gm5.reset_index()

m6 = df.iloc[(year == 2019) & (month == 6)]
gm6 = m6[['Pac(W)']].groupby([m6['Time']]).mean()
gm6 = gm6.reset_index()

m7 = df.iloc[(year == 2019) & (month == 7)]
gm7 = m7[['Pac(W)']].groupby([m7['Time']]).mean()
gm7 = gm7.reset_index()

m8 = df.iloc[(year == 2019) & (month == 8)]
gm8 = m8[['Pac(W)']].groupby([m8['Time']]).mean()
gm8 = gm8.reset_index()

m9 = df.iloc[(year == 2019) & (month == 9)]
gm9 = m9[['Pac(W)']].groupby([m9['Time']]).mean()
gm9 = gm9.reset_index()

m10 = df.iloc[(year == 2020) & (month == 10)]
gm10 = m10[['Pac(W)']].groupby([m10['Time']]).mean()
gm10 = gm10.reset_index()

m11 = df.iloc[(year == 2019) & (month == 11)]
gm11 = m11[['Pac(W)']].groupby([m11['Time']]).mean()
gm11 = gm11.reset_index()

m12 = df.iloc[(year == 2019) & (month == 12)]
gm12 = m12[['Pac(W)']].groupby([m12['Time']]).mean()
gm12 = gm12.reset_index()

fig2019 = go.Figure()

fig2019.update_layout(height=600, width=1300)

# Add traces
fig2019.add_trace(go.Scatter(x = gm9["Time"], y = gm9["Pac(W)"], mode='lines', name = 'Sep'))
fig2019.add_trace(go.Scatter(x = gm10["Time"], y = gm10["Pac(W)"], mode='lines', name = 'Oct'))
fig2019.add_trace(go.Scatter(x = gm1["Time"], y = gm1["Pac(W)"], mode='lines', name = 'Jan'))
fig2019.add_trace(go.Scatter(x = gm2["Time"], y = gm2["Pac(W)"], mode='lines', name = 'Feb'))
fig2019.add_trace(go.Scatter(x = gm3["Time"], y = gm3["Pac(W)"], mode='lines', name = 'Mar'))
fig2019.add_trace(go.Scatter(x = gm4["Time"], y = gm4["Pac(W)"], mode='lines', name = 'Apr'))
fig2019.add_trace(go.Scatter(x = gm5["Time"], y = gm5["Pac(W)"],  mode='lines',name = 'May'))
fig2019.add_trace(go.Scatter(x = gm6["Time"], y = gm6["Pac(W)"], mode='lines', name = 'Jun'))
fig2019.add_trace(go.Scatter(x = gm7["Time"], y = gm7["Pac(W)"],  mode='lines', name = 'Jul'))
fig2019.add_trace(go.Scatter(x = gm8["Time"], y = gm8["Pac(W)"], mode='lines', name = 'Aug'))
fig2019.add_trace(go.Scatter(x = gm11["Time"], y = gm11["Pac(W)"], mode = 'lines', name = 'Nov'))
fig2019.add_trace(go.Scatter(x = gm12["Time"], y = gm12["Pac(W)"], mode='lines', name = 'Dec'))

#Dados para gerar a curva típica - 2020
m1 = df.iloc[(year == 2020) & (month == 1)]
gm1 = m1[['Pac(W)']].groupby([m1['Time']]).mean()
gm1 = gm1.reset_index()

m2 = df.iloc[(year == 2020) & (month == 2)]
gm2 = m2[['Pac(W)']].groupby([m2['Time']]).mean()
gm2 = gm2.reset_index()

m3 = df.iloc[(year == 2020) & (month == 3)]
gm3 = m3[['Pac(W)']].groupby([m1['Time']]).mean()
gm3 = gm3.reset_index()

m4 = df.iloc[(year == 2020) & (month == 4)]
gm4 = m4[['Pac(W)']].groupby([m4['Time']]).mean()
gm4 = gm4.reset_index()

m5 = df.iloc[(year == 2020) & (month == 5)]
gm5 = m5[['Pac(W)']].groupby([m5['Time']]).mean()
gm5 = gm5.reset_index()

m6 = df.iloc[(year == 2020) & (month == 6)]
gm6 = m6[['Pac(W)']].groupby([m6['Time']]).mean()
gm6 = gm6.reset_index()

m7 = df.iloc[(year == 2020) & (month == 7)]
gm7 = m7[['Pac(W)']].groupby([m7['Time']]).mean()
gm7 = gm7.reset_index()

m8 = df.iloc[(year == 2020) & (month == 8)]
gm8 = m8[['Pac(W)']].groupby([m8['Time']]).mean()
gm8 = gm8.reset_index()

m9 = df.iloc[(year == 2020) & (month == 9)]
gm9 = m9[['Pac(W)']].groupby([m9['Time']]).mean()
gm9 = gm9.reset_index()

m10 = df.iloc[(year == 2020) & (month == 10)]
gm10 = m10[['Pac(W)']].groupby([m10['Time']]).mean()
gm10 = gm10.reset_index()

m11 = df.iloc[(year == 2020) & (month == 11)]
gm11 = m11[['Pac(W)']].groupby([m11['Time']]).mean()
gm11 = gm11.reset_index()

m12 = df.iloc[(year == 2020) & (month == 12)]
gm12 = m12[['Pac(W)']].groupby([m12['Time']]).mean()
gm12 = gm12.reset_index()

fig2020 = go.Figure()

fig2020.update_layout(height=600, width=1300)

# Add traces
fig2020.add_trace(go.Scatter(x = gm9["Time"], y = gm9["Pac(W)"], mode='lines', name = 'Sep'))
fig2020.add_trace(go.Scatter(x = gm10["Time"], y = gm10["Pac(W)"], mode='lines', name = 'Oct'))
fig2020.add_trace(go.Scatter(x = gm1["Time"], y = gm1["Pac(W)"], mode='lines', name = 'Jan'))
fig2020.add_trace(go.Scatter(x = gm2["Time"], y = gm2["Pac(W)"], mode='lines', name = 'Feb'))
fig2020.add_trace(go.Scatter(x = gm3["Time"], y = gm3["Pac(W)"], mode='lines', name = 'Mar'))
fig2020.add_trace(go.Scatter(x = gm4["Time"], y = gm4["Pac(W)"], mode='lines', name = 'Apr'))
fig2020.add_trace(go.Scatter(x = gm5["Time"], y = gm5["Pac(W)"],  mode='lines',name = 'May'))
fig2020.add_trace(go.Scatter(x = gm6["Time"], y = gm6["Pac(W)"], mode='lines', name = 'Jun'))
fig2020.add_trace(go.Scatter(x = gm7["Time"], y = gm7["Pac(W)"],  mode='lines', name = 'Jul'))
fig2020.add_trace(go.Scatter(x = gm8["Time"], y = gm8["Pac(W)"], mode='lines', name = 'Aug'))
fig2020.add_trace(go.Scatter(x = gm11["Time"], y = gm11["Pac(W)"], mode = 'lines', name = 'Nov'))
fig2020.add_trace(go.Scatter(x = gm12["Time"], y = gm12["Pac(W)"], mode='lines', name = 'Dec'))

#Dados para gerar a curva típica global
datall = df[['Pac(W)']].groupby([df['Time']]).mean()                            
datall = datall.reset_index()
figAll = go.Figure(data = [go.Scatter (x = (datall["Time"]), y=datall["Pac(W)"])])

figAll.update_layout(height=600, width=1300)

#Dados para gerar a tabela
#2019
gk1 = a[['Pac(W)']].groupby([a['Month']])
d = gk1.describe().T.round(2)
d.insert(0,"Measure",['count', 'mean', 'std', 'min','25%','50%','75%','max'],True)

#2020
gk = b[['Pac(W)']].groupby([b['Month']])
f = gk.describe().T.round(2)
f.insert(0,"Measure",['count', 'mean', 'std', 'min','25%','50%','75%','max'],True)

#Horário
hk = df[['Pac(W)']].groupby([df['Time']])
g = hk.describe().T.round(2)
g.insert(0,"Measure",['count', 'mean', 'std', 'min','25%','50%','75%','max'],True)

#Gerar Tabela
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

#Criando o dashboard
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.config['suppress_callback_exceptions'] = True

server = app.server

#Definido o layout
app.layout = html.Div([


        html.Div(
        id="banner",
        className="banner",
        children=[
            html.Div(
                id="banner-text",
                children=[
                    html.H5("Solar Energy"),
                    
                ],
            )],
            
        ),


        html.Div([
        dcc.Tabs(id="tabs", value='tab-1', children=[
            dcc.Tab(label='Data Analysis', value='tab-1'),
            dcc.Tab(label='Statistical Analysis', value='tab-2'),
        ]),
        html.Div(id='tabs-content')
    ])

])
        
#Definindo os callback    
@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
                html.Div([
                    dcc.Graph(figure=fig)]), 
            
            
                html.Div([
                   html.H6(["Total or Average Solar Energy per Month"], className="subtitle padded")]),

                html.Div([
                    dcc.RadioItems(id = 'ritems',
                    options=[
                        {'label': 'Sum', 'value': 'sum'},
                        {'label': 'Mean', 'value': 'mean'},
                    ],
                    value='sum',
                    labelStyle={'display': 'inline-block'}
                ), 
               
                html.Div(id='input-ritems-content')
                    
                ]), 
            
             html.Div([
                   html.H6(["BoxPlot per Period"], className="subtitle padded")]),

                html.Div([
                    dcc.RadioItems(id = 'ritems2',
                    options=[
                        {'label': 'Months', 'value': 'Months'},
                        {'label': 'Days', 'value': 'Days'},
                    ],
                    value='Months',
                    labelStyle={'display': 'inline-block'}
                ), 
               
                html.Div(id='input-ritems-content2')
                    
                ]), 
    
    
                 html.Div([
                   html.H6(["Typical Curve"], className="subtitle padded")]),

                html.Div([
                    dcc.RadioItems(id = 'ritems3',
                    options=[
                        {'label': '2019', 'value': '2019'},
                        {'label': '2020', 'value': '2020'},
                        {'label': 'All', 'value': 'All'},
                    ],
                    value='2019',
                    labelStyle={'display': 'inline-block'}
                ), 
               
                html.Div(id='input-ritems-content3')
                    
                ]) 
        
        ])

    
    elif tab == 'tab-2':
        return html.Div([
            html.Div([
                
            html.H6(["Normal Distribution per Hour"], className="subtitle padded"),
                
                
            html.H6(["Hour"], className="subtitle padded"),
            
            dcc.Dropdown(
                id='hour_selector',
                options=[{'label': i, 'value': i} for i in hours],
                value = 12
            ), 
            
            html.H6(["Minute"], className="subtitle padded"),
            
            dcc.Dropdown(
                id='minute_selector',
                options=[{'label': i, 'value': i} for i in minutes],
                value = 0
            )                  
            ]), 
                
            html.Div([
            dcc.Graph(
            id='distplot'
            )]), 
            
            html.Div([

                dcc.RadioItems(id = 'input-radio-button',
                    options=[
                        {'label': '2019', 'value': '2019'},
                        {'label': '2020', 'value': '2020'},
                        {'label': 'Hour', 'value': 'Hour'},
                    ],
                    value='2019',
                    labelStyle={'display': 'inline-block'}
                ), 
            html.Div(id='input-radio-button-content')
    ])
            
        ])


@app.callback(Output('input-ritems-content', 'children'),
              [Input('ritems', 'value')])
def update_bargraph(rit):
    if rit == 'sum':
        return html.Div([
                    dcc.Graph(figure=figPT)])
    elif rit == 'mean':
        return html.Div([
                    dcc.Graph(figure=figPM)])
    
@app.callback(Output('input-ritems-content2', 'children'),
              [Input('ritems2', 'value')])
def update_bargraph(rit2):
    if rit2 == 'Months':
        return html.Div([
                    dcc.Graph(figure=figBM)])
    elif rit2 == 'Days':
        return html.Div([
                    dcc.Graph(figure=figBD)])
    
@app.callback(Output('input-ritems-content3', 'children'),
              [Input('ritems3', 'value')])
def update_bargraph(rit3):
    if rit3 == '2019':
        return html.Div([
                    dcc.Graph(figure=fig2019)])
    if rit3 == '2020':
        return html.Div([
                dcc.Graph(figure=fig2020)]) 
    elif rit3 == 'All':
        return html.Div([
                    dcc.Graph(figure=figAll)])
    
@app.callback(
                Output('distplot','figure'),
[               Input('hour_selector', 'value'),
                Input('minute_selector', 'value')])
def update_distplot(hour_selector,minute_selector):
    
        y = df.iloc[(hour == hour_selector) & (minute == minute_selector)]

        b = y["Pac(W)"].tolist()

        hist_data = [b]
        group_labels = ['Normal Distribution'] 
    
        ff.create_distplot(hist_data, group_labels, bin_size=1000, curve_type='normal', histnorm='probability density')
  

        
        return ff.create_distplot(hist_data, group_labels, bin_size=1000, curve_type='normal', histnorm='probability density')

@app.callback(Output('input-radio-button-content', 'children'),
              [Input('input-radio-button', 'value')])
def render_content(ri):
    if ri == '2019':
        return html.Div(children=[
    html.H4(children='Descriptive Statistics'),
    generate_table(d)
])
    if ri == '2020':
        return html.Div(children=[
    html.H4(children='Descriptive Statistics'),
    generate_table(f)
])
    elif ri == 'Hour':
        return html.Div(children=[
    html.H4(children='Descriptive Statistics'),
    generate_table(g)
])

if __name__ == '__main__':
    app.run_server(debug=True)
    


In [2]:
pip freeze > Procfile

Note: you may need to restart the kernel to use updated packages.
